In [50]:
import pandas as pd
import numpy as np

In [51]:
INPUT_PATH = '../input/'
OUTPUT_PATH = '../output/'

# users = pd.read_csv(INPUT_PATH + 't_user.csv')
# orders = pd.read_csv(INPUT_PATH + 't_order.csv')
# loans = pd.read_csv(INPUT_PATH + 't_loan.csv')
# loans_sum = pd.read_csv(INPUT_PATH + 't_loan_sum.csv')
# clicks = pd.read_csv(INPUT_PATH + 't_click.csv')

# loans.loan_amount = 5**loans.loan_amount
# loans_sum.loan_sum  = 5 ** loans_sum.loan_sum

In [63]:
def load_data() :

	users = pd.read_csv(INPUT_PATH + 't_user.csv')
	orders = pd.read_csv(INPUT_PATH + 't_order.csv')
	loans = pd.read_csv(INPUT_PATH + 't_loan.csv')
	loans_sum = pd.read_csv(INPUT_PATH + 't_loan_sum.csv')
	clicks = pd.read_csv(INPUT_PATH + 't_click.csv')

	# origin price
	loans.loan_amount = 5**loans.loan_amount
	loans_sum.loan_sum  = 5 ** loans_sum.loan_sum
	orders.price = 5 ** orders.price
	orders.discount = 5 ** orders.discount 
	users.limit = 5 ** users.limit

	#loans	 
	#data to month
	loans['Date'] = pd.to_datetime(loans['loan_time'], errors='coerce')
	loans['transaction_month'] = loans['Date'].dt.month
	#fill nan with 0
	loans["loan_count"] = 1
	loans.loan_amount = loans.loan_amount.fillna(0)

	loans_new = loans.groupby(by=["uid","transaction_month"], as_index=False)["loan_amount","loan_count"].sum()
	loans_new = loans_new.pivot_table(['loan_amount',"loan_count"], ['uid'], 'transaction_month', fill_value=0)
	loans_new.reset_index( drop=False, inplace=True )
	loans_new.columns = ['{}_{}'.format(i[1], i[0]) for i in loans_new.columns]
	loans_new = loans_new.rename(index=str, columns={"_uid": "uid"})


	#orders
	orders['Date'] = pd.to_datetime(orders['buy_time'], errors='coerce')
	orders['transaction_month'] = orders['Date'].dt.month
	orders["comsume_count"] = 1

	orders["consume_amount"] = orders["price"]*orders["qty"]-orders["discount"]
	orders.consume_amount = orders.consume_amount.fillna(0)

	orders_new = orders.groupby(by=["uid","transaction_month"], as_index=False)["consume_amount","comsume_count"].sum()
	orders_new = orders_new.pivot_table(['consume_amount',"comsume_count"], ['uid'], 'transaction_month', fill_value=0)
	orders_new.reset_index( drop=False, inplace=True )
	orders_new.columns = ['{}_{}'.format(i[1], i[0]) for i in orders_new.columns]
	orders_new = orders_new.rename(index=str, columns={"_uid": "uid"})

	clicks['Date'] = pd.to_datetime(clicks['click_time'], errors='coerce')
	clicks['transaction_month'] = clicks['Date'].dt.month
	clicks["click_count"] = 1
	clicks_new = clicks.groupby(by=["uid","transaction_month"], as_index=False)["click_count"].sum()
	clicks_new = clicks_new.pivot_table('click_count', ['uid'], 'transaction_month', fill_value=0)
	clicks_new.reset_index( drop=False, inplace=True )
	clicks_new = clicks_new.rename(index=str, columns={8: "click_count_8", 9: "click_count_9", 10 : "click_count_10", 11 : "click_count_11"})

	return users, orders_new, loans_new, loans_sum, clicks_new


In [64]:
users, orders, loans, loan_sum, clicks = load_data()

In [68]:
users_orders = users.merge(orders ,how='left', on="uid")
users_orders_loans = users_orders.merge(loans ,how='left', on="uid")
user_info = users_orders_loans.merge(clicks ,how='left', on="uid")

In [75]:
list(user_info)

['uid',
 'age',
 'sex',
 'active_date',
 'limit',
 '8_comsume_count',
 '9_comsume_count',
 '10_comsume_count',
 '11_comsume_count',
 '8_consume_amount',
 '9_consume_amount',
 '10_consume_amount',
 '11_consume_amount',
 '8_loan_amount',
 '9_loan_amount',
 '10_loan_amount',
 '11_loan_amount',
 '8_loan_count',
 '9_loan_count',
 '10_loan_count',
 '11_loan_count',
 '8_click_count',
 '9_click_count',
 '10_click_count',
 '11_click_count']

In [71]:
user_info.to_csv(OUTPUT_PATH +"user_info.csv", index=False)

In [74]:
user_info = pd.read_csv(OUTPUT_PATH + 'user_info.csv')

In [77]:
for c in user_info.columns:
     user_info[c] = user_info[c].fillna(0)

In [78]:
user_info

,uid,age,sex,active_date,limit,8_comsume_count,9_comsume_count,10_comsume_count,11_comsume_count,8_consume_amount,...,10_loan_amount,11_loan_amount,8_loan_count,9_loan_count,10_loan_count,11_loan_count,8_click_count,9_click_count,10_click_count,11_click_count
0,26308,30,1,2016-02-16,15001.000001,1.0,0.0,2.0,0.0,699.000000,...,16508.000001,7005.000000,2.0,2.0,8.0,5.0,55.0,32.0,78.0,48.0
1,78209,40,1,2016-02-21,5001.000000,38.0,29.0,11.0,8.0,8391.100000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,51930,35,1,2016-04-19,25000.999999,2.0,0.0,1.0,0.0,788.000000,...,0.000000,0.000000,0.0,1.0,0.0,0.0,16.0,5.0,9.0,5.0
3,10113,25,1,2016-03-12,25000.999999,23.0,11.0,0.0,0.0,2228.800000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,47.0,74.0,25.0,8.0
4,17067,35,1,2016-02-16,15001.000001,51.0,47.0,36.0,14.0,19193.299999,...,0.000000,0.000000,0.0,0.0,0.0,0.0,11.0,19.0,10.0,22.0
5,46744,35,1,2015-12-17,15001.000001,2.0,1.0,3.0,3.0,148.500000,...,16005.000000,0.000000,1.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0
6,31747,30,2,2016-02-13,25000.999999,0.0,0.0,5.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,4.0,7.0,3.0,9.0
7,48366,30,2,2016-01-28,15001.000001,5.0,10.0,40.0,44.0,501.000000,...,0.000000,0.000000,1.0,1.0,0.0,0.0,7.0,1.0,0.0,0.0
8,49725,35,1,2016-01-28,5001.000000,1.0,0.0,8.0,2.0,19.900000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,31.0,36.0,46.0,31.0
9,31333,35,1,2016-05-17,25000.999999,8.0,5.0,1.0,2.0,618.700000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
CC = '{}_comsume_count'

In [80]:
cc_cur = CC.format(6)

In [81]:
cc_cur

'6_comsume_count'

In [82]:
month = 8
CC.format(month - 1)

'7_comsume_count'

In [83]:
INPUT_PATH = '../input/'
OUTPUT_PATH = '../output/'
CC = '{}_comsume_count'
CM = '{}_consume_amount'
LM = '{}_loan_amount'
LC = '{}_loan_count'
CKC = '{}_click_count'
CUM = '_cum'

In [115]:
def cummulate_data_by_month(df, month):
    cc_bef = CC.format(month - 1) + CUM
    cm_bef = CM.format(month - 1) + CUM
    lm_bef = LM.format(month - 1) + CUM
    lc_bef = LC.format(month - 1) + CUM
    ckc_bef = CKC.format(month - 1) + CUM

    cc_cur = CC.format(month - 1)
    cm_cur = CM.format(month - 1)
    lm_cur = LM.format(month - 1)
    lc_cur = LC.format(month - 1)
    ckc_cur = CKC.format(month - 1)

    cc_cum = CC.format(month) + CUM
    cm_cum = CM.format(month) + CUM
    lm_cum = LM.format(month) + CUM
    lc_cum = LC.format(month) + CUM
    ckc_cum = CKC.format(month) + CUM

    if month == 8:
        df[cc_cum] = 0
        df[cm_cum] = 0
        df[lm_cum] = 0
        df[lc_cum] = 0
        df[ckc_cum] = 0
    else:
        df[cc_cum] = df[cc_cur] + df[cc_bef]
        df[cm_cum] = df[cm_cur] + df[cm_bef]
        df[lm_cum] = df[lm_cur] + df[lm_bef]
        df[lc_cum] = df[lc_cur] + df[lc_bef]
        df[ckc_cum] = df[ckc_cur] + df[ckc_bef]

    return df

In [116]:
df = cummulate_data_by_month(user_info,8)

In [117]:
def get_column_by_month(df, month):
	return df.columns[df.columns.str.startswith(month)]

In [118]:
cu8 = get_column_by_month(df,'{}'.format(8))

In [119]:
cu8

Index([u'8_comsume_count', u'8_consume_amount', u'8_loan_amount',
       u'8_loan_count', u'8_click_count', u'8_comsume_count_cum',
       u'8_consume_amount_cum', u'8_loan_amount_cum', u'8_loan_count_cum',
       u'8_click_count_cum'],
      dtype='object')

In [101]:
df.columns.str.startswith('8')

array([False, False, False, False, False,  True, False, False, False,
        True, False, False, False,  True, False, False, False,  True,
       False, False, False,  True, False, False, False,  True,  True,
        True,  True,  True], dtype=bool)

In [120]:
df[cu8].head()

,8_comsume_count,8_consume_amount,8_loan_amount,8_loan_count,8_click_count,8_comsume_count_cum,8_consume_amount_cum,8_loan_amount_cum,8_loan_count_cum,8_click_count_cum
0,1.0,699.000000,2502.0,2.0,55.0,0,0,0,0,0
1,38.0,8391.100000,0.0,0.0,0.0,0,0,0,0,0
2,2.0,788.000000,0.0,0.0,16.0,0,0,0,0,0
3,23.0,2228.800000,0.0,0.0,47.0,0,0,0,0,0
4,51.0,19193.299999,0.0,0.0,11.0,0,0,0,0,0


In [121]:
df9 = cummulate_data_by_month(df,9)

In [122]:
cu9 = get_column_by_month(df,'9')

In [123]:
df[cu9].head()

,9_comsume_count,9_consume_amount,9_loan_amount,9_loan_count,9_click_count,9_comsume_count_cum,9_consume_amount_cum,9_loan_amount_cum,9_loan_count_cum,9_click_count_cum
0,0.0,0.00,3502.0,2.0,32.0,1.0,699.000000,2502.0,2.0,55.0
1,29.0,1612.30,0.0,0.0,0.0,38.0,8391.100000,0.0,0.0,0.0
2,0.0,0.00,30001.0,1.0,5.0,2.0,788.000000,0.0,0.0,16.0
3,11.0,512.20,0.0,0.0,74.0,23.0,2228.800000,0.0,0.0,47.0
4,47.0,9298.79,0.0,0.0,19.0,51.0,19193.299999,0.0,0.0,11.0


In [124]:
df10 = cummulate_data_by_month(df9,10)

In [125]:
cu10 = get_column_by_month(df10,'10')

In [126]:
df[cu10].head()

,10_comsume_count,10_consume_amount,10_loan_amount,10_loan_count,10_click_count,10_comsume_count_cum,10_consume_amount_cum,10_loan_amount_cum,10_loan_count_cum,10_click_count_cum
0,2.0,581.0,16508.000001,8.0,78.0,1.0,699.000000,6004.0,4.0,87.0
1,11.0,1772.6,0.000000,0.0,0.0,67.0,10003.399999,0.0,0.0,0.0
2,1.0,39.9,0.000000,0.0,9.0,2.0,788.000000,30001.0,1.0,21.0
3,0.0,0.0,0.000000,0.0,25.0,34.0,2741.000000,0.0,0.0,121.0
4,36.0,3790.8,0.000000,0.0,10.0,98.0,28492.089999,0.0,0.0,30.0


In [127]:
for month in [8,9,10,11,12]:
    user_info = cummulate_data_by_month(user_info, month)

In [129]:
list(user_info)

['uid',
 'age',
 'sex',
 'active_date',
 'limit',
 '8_comsume_count',
 '9_comsume_count',
 '10_comsume_count',
 '11_comsume_count',
 '8_consume_amount',
 '9_consume_amount',
 '10_consume_amount',
 '11_consume_amount',
 '8_loan_amount',
 '9_loan_amount',
 '10_loan_amount',
 '11_loan_amount',
 '8_loan_count',
 '9_loan_count',
 '10_loan_count',
 '11_loan_count',
 '8_click_count',
 '9_click_count',
 '10_click_count',
 '11_click_count',
 '8_comsume_count_cum',
 '8_consume_amount_cum',
 '8_loan_amount_cum',
 '8_loan_count_cum',
 '8_click_count_cum',
 '9_comsume_count_cum',
 '9_consume_amount_cum',
 '9_loan_amount_cum',
 '9_loan_count_cum',
 '9_click_count_cum',
 '7_comsume_count_cum',
 '7_consume_amount_cum',
 '7_loan_amount_cum',
 '7_loan_count_cum',
 '7_click_count_cum',
 '10_comsume_count_cum',
 '10_consume_amount_cum',
 '10_loan_amount_cum',
 '10_loan_count_cum',
 '10_click_count_cum',
 '11_comsume_count_cum',
 '11_consume_amount_cum',
 '11_loan_amount_cum',
 '11_loan_count_cum',
 '11_cl

In [130]:
[1]+[2]

[1, 2]

In [140]:
basic = ['uid', 'age', 'sex', 'active_date', 'limit']



def get_column_by_month(df, month):
    return list(df.columns[df.columns.str.startswith(month)])


def get_df_by_month(df, month):
    cols = get_column_by_month(df, month)
    basic = ['uid', 'age', 'sex', 'active_date', 'limit']
    all_cols = basic +cols 
    return df[all_cols]

In [143]:
df8 = get_df_by_month(user_info, '8')
df9 = get_df_by_month(user_info, '9')
df10 = get_df_by_month(user_info, '10')
df11 = get_df_by_month(user_info, '11')

In [147]:
TARGET = 'target'

In [148]:
def add_target_by_month(df, month, user_info):
    lm_next = LM.format(month + 1)
    df[TARGET] = user_info[lm_next]
    return df

In [149]:
df8 = get_df_by_month(user_info, '8')
df8 = add_target_by_month(df8, 8, user_info)
df9 = get_df_by_month(user_info, '9')
df9 = add_target_by_month(df9, 9, user_info)
df10 = get_df_by_month(user_info, '10')
df10 = add_target_by_month(df10, 10, user_info)
df11 = get_df_by_month(user_info, '11')

/Library/Python/2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [151]:
df8.head()

,uid,age,sex,active_date,limit,8_comsume_count,8_consume_amount,8_loan_amount,8_loan_count,8_click_count,8_comsume_count_cum,8_consume_amount_cum,8_loan_amount_cum,8_loan_count_cum,8_click_count_cum,target
0,26308,30,1,2016-02-16,15001.000001,1.0,699.000000,2502.0,2.0,55.0,0,0,0,0,0,3502.0
1,78209,40,1,2016-02-21,5001.000000,38.0,8391.100000,0.0,0.0,0.0,0,0,0,0,0,0.0
2,51930,35,1,2016-04-19,25000.999999,2.0,788.000000,0.0,0.0,16.0,0,0,0,0,0,30001.0
3,10113,25,1,2016-03-12,25000.999999,23.0,2228.800000,0.0,0.0,47.0,0,0,0,0,0,0.0
4,17067,35,1,2016-02-16,15001.000001,51.0,19193.299999,0.0,0.0,11.0,0,0,0,0,0,0.0


In [153]:
df9.head()

,uid,age,sex,active_date,limit,9_comsume_count,9_consume_amount,9_loan_amount,9_loan_count,9_click_count,9_comsume_count_cum,9_consume_amount_cum,9_loan_amount_cum,9_loan_count_cum,9_click_count_cum,target
0,26308,30,1,2016-02-16,15001.000001,0.0,0.00,3502.0,2.0,32.0,1.0,699.000000,2502.0,2.0,55.0,16508.000001
1,78209,40,1,2016-02-21,5001.000000,29.0,1612.30,0.0,0.0,0.0,38.0,8391.100000,0.0,0.0,0.0,0.000000
2,51930,35,1,2016-04-19,25000.999999,0.0,0.00,30001.0,1.0,5.0,2.0,788.000000,0.0,0.0,16.0,0.000000
3,10113,25,1,2016-03-12,25000.999999,11.0,512.20,0.0,0.0,74.0,23.0,2228.800000,0.0,0.0,47.0,0.000000
4,17067,35,1,2016-02-16,15001.000001,47.0,9298.79,0.0,0.0,19.0,51.0,19193.299999,0.0,0.0,11.0,0.000000


In [157]:
user_info[get_column_by_month(user_info,'9')+basic].head()

,9_comsume_count,9_consume_amount,9_loan_amount,9_loan_count,9_click_count,9_comsume_count_cum,9_consume_amount_cum,9_loan_amount_cum,9_loan_count_cum,9_click_count_cum,uid,age,sex,active_date,limit
0,0.0,0.00,3502.0,2.0,32.0,1.0,699.000000,2502.0,2.0,55.0,26308,30,1,2016-02-16,15001.000001
1,29.0,1612.30,0.0,0.0,0.0,38.0,8391.100000,0.0,0.0,0.0,78209,40,1,2016-02-21,5001.000000
2,0.0,0.00,30001.0,1.0,5.0,2.0,788.000000,0.0,0.0,16.0,51930,35,1,2016-04-19,25000.999999
3,11.0,512.20,0.0,0.0,74.0,23.0,2228.800000,0.0,0.0,47.0,10113,25,1,2016-03-12,25000.999999
4,47.0,9298.79,0.0,0.0,19.0,51.0,19193.299999,0.0,0.0,11.0,17067,35,1,2016-02-16,15001.000001


In [166]:
frames = [df8, df9, df10]
result = pd.concat(frames, axis=0)

In [168]:
list(df8)

['uid',
 'age',
 'sex',
 'active_date',
 'limit',
 '8_comsume_count',
 '8_consume_amount',
 '8_loan_amount',
 '8_loan_count',
 '8_click_count',
 '8_comsume_count_cum',
 '8_consume_amount_cum',
 '8_loan_amount_cum',
 '8_loan_count_cum',
 '8_click_count_cum',
 'target']

In [171]:
import re


In [177]:
if re.match(r"\d{1}_(.+)", '8_test_asdf'):
    print(col_name[1:])
else:
    print('dd')

NameError: name 'col_name' is not defined

In [215]:
def remove_month_rename(col_name):
    if re.match(r"(\d{1,2}_)(.+)", col_name):
        m = re.search(r"(\d{1,2}_)(.+)", col_name)
        return m.group(2)
    else:
        return col_name

In [216]:
    df8 = get_df_by_month(user_info, '8')
    df8 = add_target_by_month(df8, 8, user_info)
    df8 = df8.rename(columns=remove_month_rename)
    df9 = get_df_by_month(user_info, '9')
    df9 = add_target_by_month(df9, 9, user_info)
    df9 = df9.rename(columns=remove_month_rename)
    df10 = get_df_by_month(user_info, '10')
    df10 = add_target_by_month(df10, 10, user_info)
    df10 = df10.rename(columns=remove_month_rename)
    df11 = get_df_by_month(user_info, '11')
    df11 = df11.rename(columns=remove_month_rename)

    frames = [df8, df9, df10]
    result = pd.concat(frames)

/Library/Python/2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [214]:
m.group(2)

'adf_asdf'

In [218]:
result.head()

,uid,age,sex,active_date,limit,comsume_count,consume_amount,loan_amount,loan_count,click_count,comsume_count_cum,consume_amount_cum,loan_amount_cum,loan_count_cum,click_count_cum,target
0,26308,30,1,2016-02-16,15001.000001,1.0,699.000000,2502.0,2.0,55.0,0.0,0.0,0.0,0.0,0.0,3502.0
1,78209,40,1,2016-02-21,5001.000000,38.0,8391.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,51930,35,1,2016-04-19,25000.999999,2.0,788.000000,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,30001.0
3,10113,25,1,2016-03-12,25000.999999,23.0,2228.800000,0.0,0.0,47.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17067,35,1,2016-02-16,15001.000001,51.0,19193.299999,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0


In [220]:
df11.head()

,uid,age,sex,active_date,limit,comsume_count,consume_amount,loan_amount,loan_count,click_count,comsume_count_cum,consume_amount_cum,loan_amount_cum,loan_count_cum,click_count_cum
0,26308,30,1,2016-02-16,15001.000001,0.0,0.0,7005.0,5.0,48.0,3.0,1280.000000,22512.0,12.0,165.0
1,78209,40,1,2016-02-21,5001.000000,8.0,648.6,0.0,0.0,0.0,78.0,11775.999999,0.0,0.0,0.0
2,51930,35,1,2016-04-19,25000.999999,0.0,0.0,0.0,0.0,5.0,3.0,827.900000,30001.0,1.0,30.0
3,10113,25,1,2016-03-12,25000.999999,0.0,0.0,0.0,0.0,8.0,34.0,2741.000000,0.0,0.0,146.0
4,17067,35,1,2016-02-16,15001.000001,14.0,1357.6,0.0,0.0,22.0,134.0,32282.889999,0.0,0.0,40.0
